In [51]:
import numpy as np

In [52]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [53]:
data = load_breast_cancer()
X = data.data
y = data.target.reshape(-1, 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape,X_test.shape)

(455, 30) (114, 30)


In [54]:
class Dense:
    def __init__(self,input_dim,output_dim):
        self.weights = np.random.randn(input_dim,output_dim)*0.01
        self.biases = np.zeros((1,output_dim))

    def forward(self,X):
        self.input = X
        self.output = np.dot(X,self.weights) + self.biases
        return self.output
    
    def backward(self,d_loss_pred):
        self.d_weights = np.dot(self.input.T,d_loss_pred)
        self.d_biases = np.sum(d_loss_pred,axis=0,keepdims=True)
        d_output = np.dot(d_loss_pred,self.weights.T)
        return d_output

In [55]:
class ReLU:
    def forward(self,a):
        self.input = a
        self.output = np.maximum(0,a)
        return self.output
    
    def backward(self,d_output):
        d_relu = (self.input > 0)
        d_output_relu = d_output * d_relu
        return d_output_relu

In [56]:
class Sigmoid:
    def forward(self,logits):
        self.logits = logits
        self.prediction = 1/(1+np.exp(-logits))
        return self.prediction
    
    def backward(self,d_loss):
        d_pred = self.prediction*(1-self.prediction)
        d_loss_pred = d_loss * d_pred
        return d_loss_pred

In [57]:
class BinaryCrossEntropy:
    def forward(self,prediction,y):
        self.y_pred = prediction
        self.y_true = y
        self.epsilon = 1e-15
        loss = -np.mean(self.y_true * np.log(self.y_pred+self.epsilon) + 
            (1-self.y_true)* np.log(1-self.y_pred+self.epsilon))
        return loss
    
    def backward(self):
        d_loss = (self.y_pred-self.y_true)/((self.y_pred+self.epsilon)*(1-self.y_pred+self.epsilon))
        return d_loss

In [58]:
class GradientDescent:
    def update(self,layer,learning_rate):
        layer.weights = layer.weights - learning_rate*layer.d_weights
        layer.biases = layer.biases - learning_rate*layer.d_biases

In [73]:
class NeuralNetwork:
    def __init__(self):
        self.dense1 = Dense(30,16)
        self.activation1 = ReLU()
        self.dense2 = Dense(16,1)
        self.activation2 = Sigmoid()
        self.lossfunction = BinaryCrossEntropy()
        self.optimizer = GradientDescent()

    def forward_prop(self,X):
        a = self.dense1.forward(X)
        z = self.activation1.forward(a)
        logits = self.dense2.forward(z)
        prediction = self.activation2.forward(logits)
        return prediction
    
    def backward_prop(self,learning_rate):
        d_loss = self.lossfunction.backward()
        d_loss_pred = self.activation2.backward(d_loss)
        d_output = self.dense2.backward(d_loss_pred)
        self.optimizer.update(self.dense2,learning_rate)
        d_output_relu = self.activation1.backward(d_output)
        f_output = self.dense1.backward(d_output_relu)
        self.optimizer.update(self.dense1,learning_rate)

    def train(self,X_train,y_train,epochs,learning_rate):
        for epoch in range(epochs):
            y_pred = self.forward_prop(X_train)
            y_labels = (y_pred > 0.5)
            acc = np.mean(y_labels == y_train)
            loss = self.lossfunction.forward(y_pred,y_train)
            if (epoch+1)%100==0:
                print(f"In Epoch: {epoch+1}, Loss: {loss}, Training Accuracy: {acc*100}")
            self.backward_prop(learning_rate)

    def predict(self,X):
        y_pred = self.forward_prop(X)
        y_labels = (y_pred > 0.5)
        return y_labels

In [74]:
nn = NeuralNetwork()

In [75]:
nn.train(X_train,y_train,epochs=1000,learning_rate=0.01)

In Epoch: 100, Loss: 0.008780733757753819, Training Accuracy: 99.78021978021978
In Epoch: 200, Loss: 0.003508039519040072, Training Accuracy: 100.0
In Epoch: 300, Loss: 0.0018835949828411066, Training Accuracy: 100.0
In Epoch: 400, Loss: 0.0012009960342784236, Training Accuracy: 100.0
In Epoch: 500, Loss: 0.0008487139742320662, Training Accuracy: 100.0
In Epoch: 600, Loss: 0.0006428509004818844, Training Accuracy: 100.0
In Epoch: 700, Loss: 0.000511612847366385, Training Accuracy: 100.0
In Epoch: 800, Loss: 0.00042021420737300784, Training Accuracy: 100.0
In Epoch: 900, Loss: 0.00035501065151014114, Training Accuracy: 100.0
In Epoch: 1000, Loss: 0.0003057988845645052, Training Accuracy: 100.0


In [77]:
test_preds = nn.predict(X_test)
test_labels = (test_preds > 0.5)
acc = np.mean(test_labels == y_test)
print("Test Accuracy: ",acc*100)

Test Accuracy:  95.6140350877193
